#利用Textrank做文本摘要的核心思想很简单，和著名的网页排名算法PageRank类似：每个句子可以作为一个网络中的节点（称为节点i），与之相连的其他节点（例如节点j）会对其重要度产生一定的“贡献值”，该“贡献值”与节点j自身的重要度以及i、j之间的相似度（也可以称为连接的强度）有关，只需要对整个图进行迭代直至收敛，最后各节点的分值即是该句子的重要性，根据重要性排序后选取前k个句子即可作为摘要。

In [17]:
import jieba
import re 

In [15]:
def split_sentences(full_text):
    sents = re.split(u'[\n。]', full_text)
    sents = [sent for sent in sents if len(sent) > 0]  # 去除只包含\n或空白符的句子
    return sents

# Textrank的原始论文中，句子相似度是基于两个句子的共现词的个数计算的，在此沿用论文的公式：
# Similarity(Si,Sj)={wk|wk∈Si&wk∈Sj}/(log(Si)+log(Sj))
# 实现时，采用共现词计数进行相似度计算，输入的是每个句子的terms，以避免重复分词。代码如下：

In [4]:
def cal_sim(wordlist1, wordlist2):
    """
    给定两个句子的词列表，计算句子相似度。计算公式参考Textrank论文
    :param wordlist1:
    :param wordlist2:
    :return:
    """
    co_occur_sum = 0
    wordset1 = list(set(wordlist1))
    wordset2 = list(set(wordlist2))
    for word in wordset1:
        if word in wordset2:
            co_occur_sum += 1.0
    if co_occur_sum < 1e-12:  # 防止出现0的情况
        return 0.0
    denominator = math.log(len(wordset1)) + math.log(len(wordset2))
    if abs(denominator) < 1e-12:
        return 0.0
    return co_occur_sum / denominator

#利用networkx库创建一个graph实例，调用networkx的pagerank方法对graph实例进行处理即可，需要注意的是，networkx有三种pagerank的实现，分别是pagerank、pagerank_numpy和pagerank_scipy，从名称可以看出来它们分别采用了不同的底层实现，此处我们任选其一即可。代码如下：

In [25]:
def text_rank(sentences, num=10, pagerank_config={'alpha': 0.85, }):
    """
    对输入的句子进行重要度排序
    :param sentences: 句子的list
    :param num: 希望输出的句子数
    :param pagerank_config: pagerank相关设置，默认设置阻尼系数为0.85
    :return:
    """
    sorted_sentences = []
    sentences_num = len(sentences)#sentences分句后的句子
    wordlist = []  # 存储wordlist避免重复分词，其中wordlist的顺序与sentences对应
    for sent in sentences:
        tmp = []
        cur_res = jieba.cut(sent)
        for i in cur_res:
            tmp.append(i)
        wordlist.append(tmp)#分词后的句子列表
    graph = np.zeros((sentences_num, sentences_num))
    for x in range(sentences_num):
        for y in range(x, sentences_num):
            similarity = cal_sim(wordlist[x], wordlist[y])
            graph[x, y] = similarity
            graph[y, x] = similarity
    nx_graph = nx.from_numpy_matrix(graph)#从numpy矩阵返回图形，numpy矩阵被解释为该图的邻接矩阵。
    scores = nx.pagerank(nx_graph, **pagerank_config)  # this is a dict 核心：PageRank根据传入链接的结构计算图G中节点的排名。它最初被设计为对网页进行排名的算法。
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)#核心
    for index, score in sorted_scores:
        item = {"sent": sentences[index], 'score': score, 'index': index}
        sorted_sentences.append(item)
    return sorted_sentences[:num]#函数返回的结果中即包含了num句关键句子，可以作为组成摘要的基础。

# 由text_rank得到的前k个句子，只能表示这几个句子很重要，然而他们在逻辑上很难串联起来。如何重组织摘要，在学术界也是一大研究热点。根据不同的处理粒度（句子级、字词级）和不同的处理思路（根据语义重组还是改变现有词句的顺序），生成的摘要在阅读性上有很大的不同。
# 在此为了简便，选取最简单的，根据句子在文章中出现的顺序对text_rank结果进行重排序。代码如下：

In [6]:
def extract_abstracts(full_text, sent_num=10):
    """
    摘要提取的入口函数，并根据textrank结果进行摘要组织
    :param full_text:
    :param sent_num:
    :return:
    """
    sents = split_sentences(full_text)
    trank_res = text_rank(sents, num=sent_num)
    sorted_res = sorted(trank_res, key=lambda x: x['index'], reverse=False)
    return sorted_res

In [3]:
# 测试

In [22]:
import codecs
raw_text = codecs.open('./text.txt', 'r', 'utf8').read()

In [28]:
import math
import networkx as nx
res = extract_abstracts(raw_text, sent_num=5)#只是找出关键句子，组成摘要的基础。

In [31]:
for s in res:
    print(s['score'], s['sent'])

0.06619861502317748 ﻿传了两个月的中国公司要收购 AC 米兰的事情终于有了一个确切的消息，拥有 AC 米兰俱乐部股权的 Fininvest 公司官方正式确认正在和一家来自中国的企业商谈俱乐部股权出售事宜
0.062095949005264744 2015 年 11 月，AC 米兰老板贝卢斯科尼访华，并称就美丽之冠绿卡收购 AC 米兰一定数量的股权一事达成了合作意向，然而这件事也就此没了下文
0.06128367799191754 一年多前，曾有泰国财团为 AC 米兰开出了 5 亿欧元收购 48% 的股份的价码，这意味着当时 AC 米兰的估值为 10 亿欧元
0.06107030979584044 根据 AC 米兰官网上的数据，这家俱乐部从 2007 年开始就一直处在净亏损的状态中，2014 年的亏损额接近 1 亿欧元，更是创下了历史新高
0.062467472820868564 如果 AC 米兰真的被中国人买下来了，那么在这个买家看来，他买下来的也绝不仅仅只是一个足球俱乐部而已


# 基于sentence embedding(句嵌入)的方法

#识别一个文本的重要句子，可以看作是测度文中每个句子和全文的相似度，相似度越高的话，表示这个句子越重要。所以我们只要对全文及其分句进行sentence embedding后，计算分句表征向量和全文表征向量的cosine相似度，就可以大致抽取出重要句子。FastText相比word2vec是训练词向量升级版

In [3]:
import pandas as pd
import numpy as np
import jieba
from gensim.models import FastText

# FILE_PATH = 'news.zip'
news_df = pd.read_csv("./sqlResult_1558435.csv",encoding='gb18030')#compression='zip',
#定义分词函数
def cut(text): return ' '.join(jieba.cut(text)) 

main_content = pd.DataFrame()
main_content['title'] = news_df['title']
main_content['content'] = news_df['content'].fillna('')
main_content['tokenized_content'] = main_content['content'].apply(cut)

#训练词向量
with open('all_corpus.txt','w',encoding='utf-8') as f:
    f.write(' '.join(main_content['tokenized_content'].tolist()))

from gensim.models.word2vec import LineSentence
model = FastText(LineSentence('all_corpus.txt'),window=8,size=200,iter=10,min_count=1)
tokens = [token for line in main_content['tokenized_content'].tolist() for token in line.split()]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\MSI\AppData\Local\Temp\jieba.cache
Loading model cost 1.023 seconds.
Prefix dict has been built succesfully.
D:\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
main_content['content'].iloc[6]

'虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是今年 3 月份官方突然宣布回归中国，预示着很快就有夏普新机在中国登场了。那么，第一款夏普手机什么时候登陆中国呢？又会是怎么样的手机呢？\r\n近日，一款型号为 FS8016 的夏普神秘新机悄然出现在 GeekBench 的跑分库上。从其中相关信息了解到，这款机子并非旗舰定位，所搭载的是高通骁龙 660 处理器，配备有 4GB 的内存。骁龙 660 是高通今年最受瞩目的芯片之一，采用 14 纳米工艺，八个 Kryo 260 核心设计，集成 Adreno 512 GPU 和 X12 LTE 调制解调器。\r\n当前市面上只有一款机子采用了骁龙 660 处理器，那就是已经上市销售的 OPPO R11。骁龙 660 尽管并非旗舰芯片，但在多核新能上比去年骁龙 820 强，单核改进也很明显，所以放在今年仍可以让很多手机变成高端机。不过，由于 OPPO 与高通签署了排他性协议，可以独占两三个月时间。\r\n考虑到夏普既然开始测试新机了，说明只要等独占时期一过，夏普就能发布骁龙 660 新品了。按照之前被曝光的渲染图了解，夏普的新机核心竞争优势还是全面屏，因为从 2013 年推出全球首款全面屏手机 EDGEST 302SH 至今，夏普手机推出了多达 28 款的全面屏手机。\r\n在 5 月份的媒体沟通会上，惠普罗忠生表示：“我敢打赌，12 个月之后，在座的各位手机都会换掉。因为全面屏时代的到来，我们怀揣的手机都将成为传统手机。”\r\n'

In [4]:
from collections import Counter
token_counter = Counter(tokens)#counter作用就是在一个数组内，遍历所有元素，将元素出现的次数记下来
word_frequency = {w:counts/len(tokens) for w,counts in token_counter.items()}

In [21]:
def SIF_sentence_embedding(text,alpha=1e-4):
    global word_frequency#python中global关键字主要作用是声明变量的作用域,使用作用域之外的全局变量，则需要加global前缀
    
    max_fre = max(word_frequency.values())
    print(max_fre)
    sen_vec = np.zeros_like(model.wv['测试'])
    words = cut(text).split()
    words = [w for w in words if w in model]
    
    for w in words:
        fre = word_frequency.get(w,max_fre)
#         print(fre)
        weight = alpha/(fre+alpha)
        sen_vec += weight*model.wv[w]
#     print(sen_vec)
        
    sen_vec /= len(words)
    #skip SVD
    return sen_vec

In [23]:
# SIF_sentence_embedding(main_content['content'].iloc[6])

In [24]:
from scipy.spatial.distance import cosine

def get_corr(text,embed_fn=SIF_sentence_embedding):
    if isinstance(text,list): text = ' '.join(text)
        
    sub_sentences = split_sentences(text)
    sen_vec = embed_fn(text)
    
    corr_score = {}
    
    for sen in sub_sentences:
        sub_sen_vec = embed_fn(sen)
        corr_score[sen] = cosine(sen_vec,sub_sen_vec)
        
    return sorted(corr_score.items(),key=lambda x:x[1],reverse=True)

In [25]:
get_corr(main_content['content'].iloc[6])

0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  


0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309
0.06041265049289309


[('“我敢打赌', 1.0907226130366325),
 ('近日', 1.0278736557811499),
 ('在5月份的媒体沟通会上', 0.8668675273656845),
 ('”', 0.7993491888046265),
 ('惠普罗忠生表示', 0.7922278046607971),
 ('12个月之后', 0.7585933804512024),
 ('并于2013年退出中国市场', 0.7385215759277344),
 ('那么', 0.6637451350688934),
 ('不过', 0.6554292738437653),
 ('可以独占两三个月时间', 0.6336183249950409),
 ('说明只要等独占时期一过', 0.6292771995067596),
 ('但是今年3月份官方突然宣布回归中国', 0.6154944002628326),
 ('单核改进也很明显', 0.606799304485321),
 ('由于OPPO与高通签署了排他性协议', 0.596763551235199),
 ('从其中相关信息了解到', 0.5948291718959808),
 ('在座的各位手机都会换掉', 0.5924750566482544),
 ('八个Kryo260核心设计', 0.5923463106155396),
 ('按照之前被曝光的渲染图了解', 0.5901241600513458),
 ('又会是怎么样的手机呢', 0.574675977230072),
 ('因为全面屏时代的到来', 0.5626954436302185),
 ('但在多核新能上比去年骁龙820强', 0.533089816570282),
 ('已经完全没落', 0.5280669927597046),
 ('预示着很快就有夏普新机在中国登场了', 0.5153862535953522),
 ('集成Adreno512GPU和X12LTE调制解调器', 0.4891323447227478),
 ('采用14纳米工艺', 0.48760223388671875),
 ('配备有4GB的内存', 0.46080684661865234),
 ('那就是已经上市销售的OPPOR11', 0.44336611032485

In [8]:
import re

def split_sentences(text,p='[。.，,？：]',filter_p='\s+'):
    f_p = re.compile(filter_p)
    text = re.sub(f_p,'',text)
    pattern = re.compile(p)
    split = re.split(pattern,text)
    return split

def get_summarization(text,score_fn,sum_len):
    sub_sentences = split_sentences(text)
    ranking_sentences = score_fn(text)
    selected_sen = set()
    current_sen = ''
    
    for sen, _ in ranking_sentences:
        if len(current_sen)<sum_len:
            current_sen += sen
            selected_sen.add(sen)
        else:
            break
            
    summarized = []
    for sen in sub_sentences:
        if sen in selected_sen:
            summarized.append(sen)
    return summarized
    
def get_summarization_by_sen_emb(text,max_len=200):
    return get_summarization(text,get_corr,max_len)
    
print(''.join(get_summarization_by_sen_emb(main_content['content'].iloc[6])))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys


已经完全没落并于2013年退出中国市场但是今年3月份官方突然宣布回归中国那么又会是怎么样的手机呢近日从其中相关信息了解到八个Kryo260核心设计但在多核新能上比去年骁龙820强单核改进也很明显不过由于OPPO与高通签署了排他性协议可以独占两三个月时间说明只要等独占时期一过按照之前被曝光的渲染图了解在5月份的媒体沟通会上惠普罗忠生表示“我敢打赌12个月之后在座的各位手机都会换掉因为全面屏时代的到来”
